# 요약
1. Selinium과 BeautifulSoup을 통해 Wanted 채용공고 사이트 Crawling
2. 공고명, 직무, 업종, 유사, 직무, 회사명, 회사소개, 주요업무, 자격요건, 우대사항, 혜택 및 복지, 마감기한, 기술스택

In [ ]:
# 필요 라이브러리 다운
#!pip install selenium
#!pip install webdriver_manager
#!pip install html5lib
#!pip install IPython

In [ ]:
#패키지 호출
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup
import requests

from tqdm import tqdm
import time
import sys
import pandas as pd
import numpy as np

In [ ]:
def url_crawling():

    url = []

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    ul = soup.select_one('div.List_List_container__JnQMS > ul[data-cy="job-list"]')

    li_list = ul.find_all('li')
    for li in li_list:
        a_list = li.select('div[data-cy="job-card"] > a[class=""]')
        for a_tag in a_list:
            href = a_tag.get('href')
            url.append(href)

    return url

In [ ]:
#스크롤 함수

def scroll_down():
    # 현재 화면의 높이 가져오기
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 페이지 아래로 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # 페이지 로딩 대기
        # 스크롤 되는지 직접 확인해보고 조정
        time.sleep(5)
        # 새로운 높이 계산
        new_height = driver.execute_script("return document.body.scrollHeight")
        # 스크롤이 더 이상 되지 않으면 반복 종료
        if new_height == last_height:
            break
        # 스크롤이 가능한 높이 업데이트
        last_height = new_height

In [ ]:
#회사명, 공고명, 회사소개, 주요업무, 자격요건, 우대사항, 혜택 및 복지 추출

def detail_crawling(url_list):


    for url in tqdm(url_list):

        reponse = requests.get("https://www.wanted.co.kr" + url, headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'})
        html = reponse.text
        soup = BeautifulSoup(html, "lxml")
        soup = str(soup)

        #공고명
        title_text = soup[soup.find('"position":"') + 12 : soup.find('"reward":') - 2]
        title.append(title_text)

        #유사 직무
        yusa_jikmoo_text = soup[soup.find('"sub_categories":') + 18 : soup.find('"position":"') - 2]
        yusa_jikmoo.append(yusa_jikmoo_text)

        #회사명
        company_text = soup[soup.find('"company_name":"') + 16 : soup.find('"lang":"') - 2]
        company.append(company_text)

        #회사 소개
        introduce_text = soup[soup.find('"jd":') + 5 : soup.find('주요업무')]
        introduce.append(introduce_text)

        #주요 업무
        main_text = soup[soup.find('\\n주요업무\\n') + 8 : soup.find('\\n자격요건\\n')]
        main.append(main_text)

        #자격 요건
        q_text = soup[soup.find('\\n자격요건\\n') + 8 : soup.find('\\n우대사항\\n')]
        q.append(q_text)

        #우대사항
        treatement_text = soup[soup.find('\\n우대사항\\n') + 4 : soup.find('\\n혜택 및 복지\\n')]
        treatement.append(treatement_text)

        #혜택 및 복지 + 채용 프로세스 및 기타 정보
        welfare_text = soup[soup.find('\\n혜택 및 복지\\n') + 11 :]
        welfare.append(welfare_text)

        #업종
        upjong_text = soup[soup.find('"industry":') + 11 : soup.find(',"occupationalCategory":')]
        upjong.append(upjong_text)


        #마감기한
        due_time_text = soup[soup.find('"due_time":') + 11 : soup.find('"location":"', soup.find('"location":"') + 1) - 1]
        due_time.append(due_time_text)

        #기술스택
        driver.get("https://www.wanted.co.kr" + url)
        try:
            skill_element = driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div[1]/div[1]/div[1]/div[2]/section[1]/p[6]/div').text
            skill_list = skill_element.split('\n')
            skill_text = ', '.join(skill_list)
            skill.append(skill_text)
        except:
            skill_text = 'None'
            skill.append(skill_text)
        driver.back()

        #직무
        if i == 1634:
            jikmoo.append('머신러닝 엔지니어')
        elif i == 655:
            jikmoo.append('데이터 엔지니어')
        elif i == 1024:
            jikmoo.append('데이터 사이언티스트')
        elif i == 1025:
            jikmoo.append('빅데이터 엔지니어')
        elif i == 1022:
            jikmoo.append('BI 엔지니어')
        else:
            jikmoo.append('DBA')


    #return title, yusa_jikmoo, company, introduce, main, q, treatement, welfare, due_time, jikmoo

In [ ]:
# 지역 : 한국
# 경력 : 전체

# 머신러닝 엔지니어, 1634
# 데이터 엔지니어,   655
# 데이터 사이언티스트,  1024
# 빅데이터 엔지니어,  1025
# BI 엔지니어  1022
# DBA   10231
title = []
yusa_jikmoo = []
company = []
introduce = []
main = []
q = []
treatement = []
welfare = []
due_time = []
jikmoo = []
upjong = []
skill =[]


job = [1634, 655, 1024, 1025, 1022, 10231]
for i in tqdm(job):

    # 웹 드라이버 초기화
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    # 웹 페이지 접속
    driver.get(f'https://www.wanted.co.kr/wdlist/518/{i}?country=kr&job_sort=job.latest_order&years=-1&locations=all')

    # 스크롤 함수 실행
    scroll_down()

    # 크롤링 코드 작성
    url = url_crawling()
    detail_crawling(url)

    # 웹 드라이버 종료
    driver.quit()


df = pd.DataFrame({
    "공고명": title,
    "직무" : jikmoo,
    "업종" : upjong,
    "유사 직무" : yusa_jikmoo,
    "회사명": company,
    "회사소개" : introduce,
    "주요업무": main,
    "자격요건" : q,
    "우대사항" : treatement,
    "혜택 및 복지" : welfare,
    "마감기한": due_time,
    "기술스택" : skill
})

 18%|██████████████▍                                                                  | 64/359 [02:58<18:24,  3.74s/it]


 36%|████████████████████████████▉                                                   | 130/359 [06:26<10:58,  2.88s/it]


 55%|███████████████████████████████████████████▋                                    | 196/359 [09:28<06:45,  2.49s/it]


 73%|██████████████████████████████████████████████████████████▍                     | 262/359 [12:44<05:03,  3.12s/it]


 91%|█████████████████████████████████████████████████████████████████████████       | 328/359 [16:06<02:14,  4.35s/it]


...

 14%|███████████▍                                                                     | 22/155 [01:12<06:44,  3.04s/it]


 57%|█████████████████████████████████████████████▉                                   | 88/155 [04:40<03:47,  3.39s/it]


 99%|███████████████████████████████████████████████████████████████████████████████▍| 154/155 [08:05<00:02,  2.91s/it]


 51%|██████████████████████████████████████████                                        | 40/78 [02:03<01:51,  2.94s/it]


100%|█████████████████████████████████████████████████████████████████████████████████| 6/6 [1:06:09<00:00, 661.58s/it]


In [ ]:
df = pd.DataFrame({
    "공고명": title,
    "직무" : jikmoo,
    "업종" : upjong,
    "유사 직무" : yusa_jikmoo,
    "회사명": company,
    "회사소개" : introduce,
    "주요업무": main,
    "자격요건" : q,
    "우대사항" : treatement,
    "혜택 및 복지" : welfare,
    "마감기한": due_time
})

In [ ]:
df

,공고명,직무,업종,유사 직무,회사명,회사소개,주요업무,자격요건,우대사항,혜택 및 복지,마감기한,기술스택
0,"AI, ML Engineer 박사급",머신러닝 엔지니어,"""IT, 컨텐츠""","""Data Engineer"",""데이터 엔지니어"",""Machine Learning E...",플리토,"""플리토는 유수의 글로벌 IT 기업들과 함께 '언어 장벽 없는 세계'를 만들어 가고...",- 기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니...,- AI / ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분\n - M...,사항\n- 모델을 직접 만들었거나 공개된 모델을 fine-tuning해서 성능을 향...,"- 상호 존중하는 문화\n : 직급 없는 \""님\"" 호칭 및 경어 사용, 자율 ...","""2023-11-27T00:00:00""","Github, Android, Firebase, Kotlin, Coroutine"
1,AI 컴파일러 개발자 (1년 이상),머신러닝 엔지니어,"""IT, 컨텐츠""","""Machine Learning Engineer"",""머신러닝 엔지니어""",티에스엔랩,"""실시간 AI/실시간 통신 전문 기업 TSN Lab에서 네트워크 AI 개발자를 모십...",[어떤 팀에서 일 하나요?]\n• AI팀에서 일 합니다. AI팀은 AI 모델 만들고...,[어떤 사람을 채용하나요?]\n• 컴파일러 관련 지식이 있어야 합니다.\n• LLV...,,[연봉은 어떻게 되나요?]\n• 경력자의 경우 기존의 경력과 동일한 분야인 경우 기...,null,None
2,LLM 연구원 (자연어 처리),머신러닝 엔지니어,"""IT, 컨텐츠""","""Machine Learning Engineer"",""머신러닝 엔지니어""",딥노이드(Deepnoid),"""LLM 연구원 (자연어 처리)\n\n\n",LLM 모델 연구 및 고도화\n\nLLM 연구의 서비스화를 위한 개발 지원\n,"경력 3년 이상 또는 관련 학과 석사 학위 소지자\n\nPyTorch, Tensor...","사항\n인공지능 관련 대회(해커톤, 캐글, 그랜드챌린지 등)의 출전 경험 및 입상\...",1. 유연 근무제\n2. 자유 복장\n3. 최신 장비\n4. 수평적 조직\n5. 도...,"""2024-02-29T00:00:00""","C#, C, C++, WPF"
3,AI R\u0026D 팀장,머신러닝 엔지니어,"""IT, 컨텐츠""","""Machine Learning Engineer"",""머신러닝 엔지니어""",딥노이드(Deepnoid),"""AI R\u0026D 팀장\n\n\n",sLLM 모델 연구개발 및 서비스 고도화 \n\nsLLM 기반의 서비스 모델 개발 ...,AI 관련학과 박사학위 소지자 또는 이에 준하는 경력 보유자\n\nLLM 관련 최신...,"사항\nAI 관련 Top tier 학회(NeurIPS, CVPR, AAAI 등) 논...",1. 유연 근무제\n2. 자유 복장\n3. 최신 장비\n4. 수평적 조직\n5. 도...,"""2024-02-29T00:00:00""","C#, C, C++, WPF"
4,소프트웨어 엔지니어,머신러닝 엔지니어,"""IT, 컨텐츠""","""Web Developer"",""웹 개발자"",""Machine Learning Engi...","스타인펠드(Steinfeld,Inc.)","""Steinfeld.co는 치과 보철치료에 필요한 보철물을 더 효율적으로 제공하기 ...",• AI 개발\n보철물 자동 디자인을 위한 AI 모델 개발\n보철물 자동 디자인 서...,• python 또는 TypeScript 로 개발 경험 있으신분\n• Open mi...,사항\n• 성장하는 스타트업에 관심 있으신 분\n,• 경쟁력 있는 연봉\n - 협의를 통해 충분히 경쟁력 있는 수준의 연봉을 제...,null,"Git, Firebase, Google Cloud Platform, Pytorch,..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1143,DBA,DBA,"""IT, 컨텐츠""","""Database Administrator"",""DBA""",이엘파크,"""이엘파크 플랫폼사업부문 DBA 채용\n\n\n","• DB 분석/설계/구축/운영 및 관리\n• DB 성능분석, 최적화(Tuning) ...","• Mysql, MariaDB, Aurora 중 1개 이상의 관리 및 운영\n","사항\n• AWS 클라우드 플랫폼에 대한 DB 운영\n• AWS RDS, Auror...","• 유연근무, 연차 촉진, 인센티브\n• 식비지원, 경조사 지원, 명절 선물\n• ...",null,"React, HTML, JavaScript"
1144,"데이터베이스 복제 솔루션 개발(C/C++, Linux/Unix)",DBA,"""IT, 컨텐츠""","""C,C++ Developer"",""C,C++ 개발자"",""Database Admini...",아크데이타,"""아크데이타는 정보의 효율적인 저장과 가공이 기업 경쟁력의 중요한 초석이 되는 사회...",• 데이터베이스 변경 로그 추출\n• 데이터 저장 및 추출\n• 데이터베이스\n,• 신입 or 경력\n• C / C++\n,사항\n• 데이터베이스에 대한 전반적인 이해도가 높으신 분\n• 데이터베이스 기반 ...,• 회사와 함께 나도 성장합니다. 스톡옵션 부여\n• 사회 초년생을 위한 청년 내일...,null,"Git, Hadoop, Linux, MySQL, Oracle, C / C++, Py..."
1145,[핀트] DBA,DBA,"""금융""","""Data Engineer"",""데이터 엔지니어"",""Security Engineer""...",디셈버앤컴퍼니(핀트),"""[국내 최초 모바일 투자일임 서비스 핀트를 소개합니다]\n\n2019년 4월, 국...",• On-Premise 기반 환경의 데이터베이스를 구축하고 운영합니다.\n• 클라우...,"• 3년 이상의 DBA 경험이(MySQL, PostgreSQL, Maria DB) ...","사항\n• 증권, 카드, 뱅킹 관련 핀테크 산업군의 경험이 있는 분\n• 자동화 작...",최고의 퍼포먼스를 내기 위해 몰입과 성장할 수 있는 업무 환경을 만들어 가고 있습니...,null,"Hadoop, Spark, SQL, Data Analysis, Tableau, Am..."
1146,DBA,DBA,"""금융""","""Back-end Engineer"",""서버 개발자"",""Software Enginee...",핀다(FINDA),"""어렵고 복잡한, 그렇지만 우리 일상 생활에 깊숙이 관여하고 있는 금융.\n핀다는 ...",핀다 DBA는 아키텍처를 구성하고 Database를 안정적으로 운영합니다.\n• 데...,"• MYSQL DB 운영 경험이 3년 이상 있는 분\n• DB 보안, 백업에 대한 ...","사항\n• 서비스 DB 설계 및 구축 경험 있는 분\n• 플랫폼 DB 운영, 아카이...",역할에 맞는 보상을 제공합니다.\n• 적극적인 스톡옵션 제도를 운영합니다.\n• 능...,null,None


In [ ]:
df

,공고명,직무,업종,유사 직무,회사명,회사소개,주요업무,자격요건,우대사항,혜택 및 복지,마감기한
0,"AI, ML Engineer 박사급",머신러닝 엔지니어,"""IT, 컨텐츠""","""Data Engineer"",""데이터 엔지니어"",""Machine Learning E...",플리토,"""플리토는 유수의 글로벌 IT 기업들과 함께 '언어 장벽 없는 세계'를 만들어 가고...",- 기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니...,- AI / ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분\n - M...,사항\n- 모델을 직접 만들었거나 공개된 모델을 fine-tuning해서 성능을 향...,"- 상호 존중하는 문화\n : 직급 없는 \""님\"" 호칭 및 경어 사용, 자율 ...","""2023-11-27T00:00:00"""
1,AI 컴파일러 개발자 (1년 이상),머신러닝 엔지니어,"""IT, 컨텐츠""","""Machine Learning Engineer"",""머신러닝 엔지니어""",티에스엔랩,"""실시간 AI/실시간 통신 전문 기업 TSN Lab에서 네트워크 AI 개발자를 모십...",[어떤 팀에서 일 하나요?]\n• AI팀에서 일 합니다. AI팀은 AI 모델 만들고...,[어떤 사람을 채용하나요?]\n• 컴파일러 관련 지식이 있어야 합니다.\n• LLV...,,[연봉은 어떻게 되나요?]\n• 경력자의 경우 기존의 경력과 동일한 분야인 경우 기...,null
2,LLM 연구원 (자연어 처리),머신러닝 엔지니어,"""IT, 컨텐츠""","""Machine Learning Engineer"",""머신러닝 엔지니어""",딥노이드(Deepnoid),"""LLM 연구원 (자연어 처리)\n\n\n",LLM 모델 연구 및 고도화\n\nLLM 연구의 서비스화를 위한 개발 지원\n,"경력 3년 이상 또는 관련 학과 석사 학위 소지자\n\nPyTorch, Tensor...","사항\n인공지능 관련 대회(해커톤, 캐글, 그랜드챌린지 등)의 출전 경험 및 입상\...",1. 유연 근무제\n2. 자유 복장\n3. 최신 장비\n4. 수평적 조직\n5. 도...,"""2024-02-29T00:00:00"""
3,AI R\u0026D 팀장,머신러닝 엔지니어,"""IT, 컨텐츠""","""Machine Learning Engineer"",""머신러닝 엔지니어""",딥노이드(Deepnoid),"""AI R\u0026D 팀장\n\n\n",sLLM 모델 연구개발 및 서비스 고도화 \n\nsLLM 기반의 서비스 모델 개발 ...,AI 관련학과 박사학위 소지자 또는 이에 준하는 경력 보유자\n\nLLM 관련 최신...,"사항\nAI 관련 Top tier 학회(NeurIPS, CVPR, AAAI 등) 논...",1. 유연 근무제\n2. 자유 복장\n3. 최신 장비\n4. 수평적 조직\n5. 도...,"""2024-02-29T00:00:00"""
4,소프트웨어 엔지니어,머신러닝 엔지니어,"""IT, 컨텐츠""","""Web Developer"",""웹 개발자"",""Machine Learning Engi...","스타인펠드(Steinfeld,Inc.)","""Steinfeld.co는 치과 보철치료에 필요한 보철물을 더 효율적으로 제공하기 ...",• AI 개발\n보철물 자동 디자인을 위한 AI 모델 개발\n보철물 자동 디자인 서...,• python 또는 TypeScript 로 개발 경험 있으신분\n• Open mi...,사항\n• 성장하는 스타트업에 관심 있으신 분\n,• 경쟁력 있는 연봉\n - 협의를 통해 충분히 경쟁력 있는 수준의 연봉을 제...,null
...,...,...,...,...,...,...,...,...,...,...,...
370,소프트웨어 개발 (신입),데이터 엔지니어,"""IT, 컨텐츠""","""Data Engineer"",""데이터 엔지니어"",""Development Manage...",브이엠에스솔루션스,"""브이엠에스 솔루션스는 시뮬레이션 기반 SCM 솔루션 공급 전문 기업으로, \n독자...",[이런 일을 합니다]\n\n• Planning \u0026 Scheduling 관련...,• 경력 ： 신입\n• 학력 ： 학사 이상\n• C# 언어를 이용한 Windows ...,사항\n• 정보통신관련 자격증 보유자\n• MES/ERP/SCM 관련업무 경험자\n,[복리후생]\n\n[주거 지원]\n• 직원의 주택자금 대출 이자를 지원합니다.\n•...,"""2023-11-30T00:00:00"""
371,Data Engineer (BI 개발자),데이터 엔지니어,"""IT, 컨텐츠""","""Data Engineer"",""데이터 엔지니어"",""BI Engineer"",""BI 엔...",인포유앤컴퍼니,"""\"" Data, Data, and more data : Bring Insight\...",• Power BI를 이용한 분석 보고서 개발\n• 데이터의 기본 이해를 통한 모델...,"• Tableau, Power BI 등 시각화 개발 2년차\n• 데이터에 대한 이해도\n",사항\n• Public Cloud 경험이 있으신 분\n• ERP System 개발/...,• 성과급(상반기/하반기 인센티브) 지급\n• 능력에 따른 급여 인상 체계\n- 연...,null
372,DA(Data Modeler),데이터 엔지니어,"""IT, 컨텐츠""","""Data Engineer"",""데이터 엔지니어"",""Database Administr...",인포유앤컴퍼니,"""\"" Data, Data, and more data : Bring Insight\...",• 대용량의 데이터를 데이터 성격에 맞게 분석/설계\n• 효과적인 DataMart를...,• 마트 설계 업무 3년 이상의 경력 보유자\n• SQL 툴 1개 이상으로 2년 이...,사항\n• 클라우드 기반의 데이터 플랫폼 구축 및 운영 경험이 있으신 분\n• 데이...,• 성과급(상반기/하반기 인센티브) 지급\n• 능력에 따른 급여 인상 체계\n- 연...,null
373,LLM AI 연구개발자,데이터 엔지니어,"""IT, 컨텐츠""","""Data Engineer"",""데이터 엔지니어"",""C,C++ Developer"",""...",엔쓰리엔(N3N),"""• 엔쓰리엔 신규사업 반려로봇 개발에 따른 LLM AI 개발자를 채용합니다.\n\n\n",• 생성형 인공지능 연구 개발\n• AI 기반 NLP 연구 개발\n• 초거대 AI기...,• python 사용 가능자\n• 데이터 수집/분석/처리방법에 대해 분석 가능자\n...,사항\n•석박사 우대\n,• 시차출근제\n• 교육비 및 도서구입비 지원\n• 팀 운영비 지원\n• 희망장비 ...,null


In [ ]:
df.isnull().sum()

공고명        0
직무         0
업종         0
유사 직무      0
회사명        0
회사소개       0
주요업무       0
자격요건       0
우대사항       0
혜택 및 복지    0
마감기한       0
기술스택       0
dtype: int64

In [ ]:
df['기술스택'].value_counts()

None                                                                                                     365
Pytorch, Tensorflow, Python                                                                               16
Python                                                                                                    15
JavaScript, Node.js, Blockchain, Docker, NodeJS                                                           14
GitLab, CSS, TypeScript, React.js, Next.js                                                                12
                                                                                                        ... 
C#, Java                                                                                                   1
DevOps                                                                                                     1
Django, React, JavaScript, Python, TypeScript, FastAPI                                                     1
Git, Firebase, Goog

In [ ]:
df.to_csv('wanted.csv', index=False)